# Raw BigEarthNet Data

After understanding where the patches come from and how the patches were annotated, the following section will present and discuss the files inside the archives.

## BigEarthNet-S2

The general contents of the BigEarthNet-S2 archive looks as follows:

In [ ]:
# remove-input

from rich import print
from rich.tree import Tree
from rich.markup import escape
from rich.text import Text
from pathlib import Path
from pydantic import validate_arguments, DirectoryPath
import natsort
from myst_nb import glue

ben_s2_path = Path("./BigEarthNet-S2-Example")

def _first_dir_natsort_key(path: Path) -> str:
    """Directories should come first"""
    prefix = 0 if path.is_dir() else 1
    return f"{prefix}{path}"

def _walk_ben_directory(directory: Path, tree: Tree):
    """
    Recursively build a BigEarthNet directory
    """
    # Sort dirs first
    paths = sorted(
        Path(directory).iterdir(), key=natsort.natsort_keygen(_first_dir_natsort_key)
    )
    for path in paths:
        if path.name.startswith("."):
            continue
        if path.is_dir():
            branch = tree.add(
                # f":open_file_folder: [link file://{path}]{escape(path.name)}"
                f":open_file_folder: [bold]{escape(path.name)}"
            )
            _walk_ben_directory(path, branch)
        else:
            text_filename = Text(path.name)
            text_filename.highlight_regex(r".*_\d\d?_", "dim")
            text_filename.highlight_words(["labels_metadata.json"], "bold")
            text_filename.highlight_regex(r"(?<=_).*.tif", "bold")
            # text_filename.stylize(f"link file://{path}")
            icon = "🗺️ " if path.suffix == ".tif" else "📄 "
            tree.add(Text(icon) + text_filename)


@validate_arguments
def walk_ben_directory(directory: DirectoryPath) -> Tree:
    directory = directory.resolve()
    tree = Tree(
        # f":open_file_folder: [link file://{directory}]{directory.name}"
        f":open_file_folder: [bold]{directory.name}"
    )
    _walk_ben_directory(directory, tree)
    return tree

walk_ben_directory(ben_s2_path)

With the following conventions:
- Each folder coressponds to a single patch
- The `patch_name` is encoded as the name of the folder
- Each patch folder contains a [GeoTIFF](https://en.wikipedia.org/wiki/GeoTIFF) file for each of the 12 bands.
  - The name of the GeoTIFF file is encoded as `<patch_name>_<band>.tif`.
- The [JSON](https://en.wikipedia.org/wiki/JSON) file, named `<patch_name>_labels_metadata.json`, contains the metadata

The prettified contents of a metadata file is:

In [ ]:
# remove-input

from rich import print_json
from copy import copy
import json

ben_s2_json_file_paths = list(Path(ben_s2_path).rglob("*.json"))
ben_s2_json_fp = ben_s2_json_file_paths[0]
text = ben_s2_json_fp.read_text()
j = json.loads(text)
simple_j = copy(j)
simple_j["projection"] = simple_j["projection"][:75] + "..."

print_json(data=simple_j)

- `labels`: Lists the older CLC Level-3 nomenclature labels of the patch
<!-- TODO: Cross-Ref CLC Level-3 and tile -->
- `tile_source`: Shows the source [tile] that was further processed with [sen2cor](https://step.esa.int/main/snap-supported-plugins/sen2cor/) to generate the atmospherically corrected [L2A](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/product-formatting) product tile
- `acquisition_date`: Encodes the acquisition date of the tile in the `YYYY-MM-DD hh:mm:ss` format
- `coordinates`: Encodes the upper left x/y (`ulx`/`uly`) and lower right x/y (`lrx`/`lry`) coordinates of the patch
- `projection`: Relates the values of the `coordinates` to the given coordinate reference systems (CRS)

The unshorted (prettified) `projection` entry looks as follows:

In [ ]:
# remove-input
import pyproj

print(pyproj.CRS.from_user_input(j["projection"]).to_wkt(pretty=True))

The `projection` entry encodes the CRS information in the [WKT](https://en.wikipedia.org/wiki/Well-known_text_representation_of_coordinate_reference_systems) format.
For most use-cases, it is sufficient to know, that the combination of the CRS and `coordinates` values define the exact location of a patch.
For more details about what coordinate reference systems are feel free to take a look at one of the following introductory courses:
- [earthdatascience earth-analytics course](https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/intro-to-coordinate-reference-systems/)
- [gisbooklet](https://bookdown.org/tep/gisbooklet/introduction-to-coordinate-reference-system.html)

## Metadata Discussion

In [ ]:
# remove-input
from bigearthnet_common.gdf_builder import get_gdf_from_patch_dir

gdf = get_gdf_from_patch_dir(ben_s2_path)
gdf["tile_source"] = gdf["tile_source"].str.wrap(15, break_long_words=True)
gdf["name"] = gdf["name"].str.wrap(15, break_long_words=True)
gdf_style = gdf.style.set_properties(**{
    'white-space': 'pre-wrap',
})
# glue("ben-s2-gdf", gdf_style)
glue("ben-s2-gdf", gdf_style, display=False)

The advantages of having a JSON metadata file in every patch folder are:
1. JSON is well known data format and has excellent library support
2. Is human-readable (not a binary format)
3. Locating the metadata _next_ to the images allows the end-user to easily select subsets of the archive without having to deal with the metadata separately
   - Copying the patches of interest will always include the metadata

The main disadvantages is that each archive (~80GB) has to be downloaded and that it is not easy to perform statistical analysis.
The metadata files have to be parsed and converted into a common data structure first.
Usually, the metadata is converted into a tabular format to allow the use of data analysis tools, such as [pandas](https://pandas.pydata.org/), or the geographical extension, [geopandas](https://geopandas.org/en/stable/).

Instead of re-writing another parsing script, we recommend to use one of the pre-converted files:
- BigEarthNet-S2
   - [raw_ben_gdf.parquet](https://tubcloud.tu-berlin.de/s/gxp8KGk8TfcH6rR)
      - The original parquet file that is produced by parsing all metadata files and projecting to a common CRS
   - [extended_ben_gdf.parquet](https://tubcloud.tu-berlin.de/s/5cstd7EJRB8kCyL)
      - An extended version of the `raw_ben_gdf.parquet` file with additional metadata:
        - 19-class nomenclature
        - Covered by seasonal snow
        - Covered by clouds or shadows
        - Original split
        - Country
        - Season
   - [final_ben.parqet](https://tubcloud.tu-berlin.de/s/wtqZQKawLaBZZY8)
     - The recommended subset of `extended_ben_gdf.parquet`, where no patch is covered by snow, clouds or shadows and every patch has at least one target label in the 19-class nomenclature

Another option is to use an open-source library called [bigearthnet_common](https://kai-tub.github.io/bigearthnet_common/) to convert the metadata into a tabular format:

:::{dropdown} BigEarthNet-S2-Example Metadata in tabular form
:::{glue:} ben-s2-gdf
:::


:::{important}
Instead of writing another metadata loading script:
- Download one of the pre-converted files or
- Use the [bigearthnet_common](https://kai-tub.github.io/bigearthnet_common/) tool to convert the metadata into a tabular format
:::